In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
static = spark.read.parquet("hdfs://namenode:9000/data/articles.parquet")
dataSchema = static.schema

In [ ]:
static.printSchema()

In [ ]:
static.show(4)

In [ ]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.parquet("hdfs://namenode:9000/data/articles.parquet")

In [ ]:
activityCounts = streaming.groupBy("label").count()

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [ ]:
activityQuery = activityCounts.writeStream.queryName("label_counts")\
.format("memory").outputMode("complete")\
.start()

In [ ]:
activityQuery.awaitTermination()

In [ ]:
list(spark.streams.active)

In [ ]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)